# Generate landscape and elevation points for a dataframe that already contains the Veðurstofu measurement and Carra reanalysis

The type of landscape distribution to be generated can be changed by changing the imports from utils.elevation

In [1]:
from utils.elevation import generateLandscapeDistribution2Sectors, generateElevationDistribution, findLandscapeElevation
from utils.util import is_laptop
from datetime import date

import pandas as pd, rasterio, os

In [2]:
folder_path = 'D:/Skoli/Mastersverkefni/lokaverkefni_vel/data/' if is_laptop() else 'E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/data/'
merged_nailstripped_path = folder_path + 'NailStripped/' + max(os.listdir(folder_path + 'NailStripped/'), key = lambda f: os.path.getmtime(folder_path + 'NailStripped/' + f))
elevationPath = folder_path + "Elevation/IslandsDEMv1.0_20x20m_isn93_zmasl.tif"
outputPath = folder_path + "NailStripped_W_Elevation/" + f'merged_stripped_elevation_{date.today()}.feather'

In [3]:
df = pd.read_feather(merged_nailstripped_path)

with rasterio.open(elevationPath) as dataset:
    elevation = dataset.read(1)
    index = dataset.index
    transform = dataset.transform

In [4]:
def addPointElevation(row, transform, index, elevation):
    X, Y = row.X, row.Y
    return findLandscapeElevation((X,Y), transform, index, elevation)

In [5]:
df['station_elevation'] = df.apply(addPointElevation, args = (transform, index, elevation), axis = 1)
df['landscape_points'] = df.apply(generateLandscapeDistribution2Sectors, axis = 1)
df['elevations']  = df.apply(generateElevationDistribution, args = (transform, index, elevation), axis = 1)

In [6]:
df.to_feather(outputPath)